#### Importações

In [93]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [94]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [95]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [99]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [100]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,122.251200,22.307377,20.572579,99.000,99.000,107.500,1.020000e+02,1.160000e+02,1.170000e+02,142.500000,1.430000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,19041.621100,5472.264453,3427.966943,10054.000,10196.000,16530.500,1.671100e+04,1.858400e+04,1.914450e+04,21182.000000,2.157225e+04,2.764700e+04,2.729000e+04
installation_date,38.857143,38.490500,5.639993,4.739510,31.000,31.000,34.500,3.400000e+01,4.100000e+01,3.900000e+01,42.500000,4.300000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.479200,0.786796,0.759753,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.705200,1.511858,0.929612,4.000,4.000,8.000,7.000000e+00,8.000000e+00,7.000000e+00,8.000000,7.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,15.452600,10.106575,8.013562,5.000,5.000,8.500,8.000000e+00,2.000000e+01,1.500000e+01,23.000000,2.200000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,136.005667,18.993420,16.856972,114.000,114.000,122.750,1.188088e+02,1.360000e+02,1.351590e+02,150.500000,1.520795e+02,1.660000e+02,1.657920e+02
distance_to_coast (km),218.571429,218.948300,66.271807,66.631817,120.000,120.000,180.000,1.400000e+02,2.400000e+02,2.560000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.478800,0.534522,0.499575,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000007,0.000010,0.000009,0.000,0.000,0.000,6.203353e-10,7.800000e-08,2.043008e-07,0.000014,1.611396e-05,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [101]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [102]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [103]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [104]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [105]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [106]:
df_gan_limpo = remover_outliers_iqr(df_gan)

4991


In [107]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (5009, 25))

In [108]:
df_gan_limpo.duplicated().sum()

0

#### Comparando novamente as bases de dados

In [109]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [110]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,117.675584,22.307377,19.827248,99.000,99.000,107.500,1.000000e+02,1.160000e+02,1.080000e+02,142.500000,1.380000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,20129.504292,5472.264453,2716.090959,10054.000,12427.000,16530.500,1.813200e+04,1.858400e+04,2.006700e+04,21182.000000,2.216100e+04,2.764700e+04,2.670100e+04
installation_date,38.857143,39.009782,5.639993,3.601188,31.000,31.000,34.500,3.600000e+01,4.100000e+01,3.900000e+01,42.500000,4.200000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.391895,0.786796,0.704520,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.000000,1.511858,0.000000,4.000,7.000,8.000,7.000000e+00,8.000000e+00,7.000000e+00,8.000000,7.000000e+00,8.000000e+00,7.000000e+00
number_of_piles,17.142857,12.992813,10.106575,7.167743,5.000,5.000,8.500,7.000000e+00,2.000000e+01,1.100000e+01,23.000000,1.800000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,131.355897,18.993420,16.843002,114.000,114.000,122.750,1.157780e+02,1.360000e+02,1.244350e+02,150.500000,1.478690e+02,1.660000e+02,1.650820e+02
distance_to_coast (km),218.571429,233.228389,66.271807,59.986049,120.000,120.000,180.000,1.780000e+02,2.400000e+02,2.700000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.639848,0.534522,0.480092,0.000,0.000,0.000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000004,0.000010,0.000007,0.000,0.000,0.000,5.072355e-11,7.800000e-08,1.633915e-08,0.000014,5.172527e-06,2.300000e-05,2.299925e-05


In [111]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),2.177371e+00,1.734797e+00,0.000,5.500000e+00,-1.000000e+00,-0.500000,1.000
weight (t),-2.259068e+02,2.044298e+03,-142.000,-1.805000e+02,-5.605000e+02,-390.250000,357.000
installation_date,3.666429e-01,9.004831e-01,0.000,5.000000e-01,2.000000e+00,-0.500000,1.000
type_of_production (1 oil and gas; 2 oil; 3 gas),9.222857e-02,2.704249e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
number_of_legs,7.233714e-01,5.822454e-01,0.000,1.000000e+00,1.000000e+00,1.000000,0.000
number_of_piles,1.690257e+00,2.093013e+00,0.000,5.000000e-01,5.000000e+00,1.000000,1.000
height_of_jacket_or_sub-structure (m),1.494333e+00,2.136448e+00,0.000,3.941250e+00,8.410000e-01,-1.579500,0.208
distance_to_coast (km),-3.768714e-01,-3.600099e-01,0.000,4.000000e+01,-1.600000e+01,-17.000000,0.000
risk_to_other_users-complete,-5.022857e-02,3.494715e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
risk_to_other_users-partial,6.727060e-07,3.070633e-07,0.000,-6.203353e-10,-1.263008e-07,-0.000002,0.000


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [122]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [112]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.53,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [113]:
df_gan_limpo.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,150,17381,42,2,7,20,158.344,194,0,1.575815e-05,0.093,0.067,441426,545478,32618,37688,0.720,0.390,0.983,0.993,0.904,0.857,0.916,0.978,0.609
1,108,17668,37,1,7,10,126.002,274,1,1.333454e-07,0.162,0.093,979886,762796,73655,48009,0.277,0.018,0.990,0.959,0.918,0.696,0.558,0.992,0.604
4,148,21662,43,3,7,25,156.532,138,0,1.984997e-05,0.093,0.067,462282,588855,36172,45347,0.509,0.458,0.874,0.832,0.550,0.701,0.769,0.973,0.624
6,134,20737,40,1,7,24,149.745,258,0,1.290168e-05,0.030,0.015,330738,482090,35206,35644,0.614,0.433,0.842,0.978,0.571,0.705,0.851,0.995,0.586
8,99,23559,43,1,7,6,114.333,281,1,6.854534e-13,0.318,0.156,1178227,894785,91909,63873,0.001,0.000,1.000,0.978,0.936,0.603,0.553,1.000,0.560


In [124]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# Criar o metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=A)
metadata_dict = metadata.to_dict()

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 593.23it/s]|
Column Shapes Score: 72.44%

(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:00<00:00, 355.88it/s]|
Column Pair Trends Score: 93.54%

Overall Score (Average): 82.99%



In [116]:
report.get_score()

0.8298845889179576

In [117]:
report.get_properties()

,Property,Score
0,Column Shapes,0.724352
1,Column Pair Trends,0.935418


In [118]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,installation_date,KSComplement,0.827071
1,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.886829
2,number_of_legs,KSComplement,0.194757
3,number_of_piles,KSComplement,0.746271
4,height_of_jacket_or_sub-structure (m),KSComplement,0.820657
5,risk_to_other_users-complete,KSComplement,0.949771
6,risk_to_other_users-partial,KSComplement,0.642629
7,risk_to_personnel-complete,KSComplement,0.771214
8,risk_to_personnel-partial,KSComplement,0.833686
9,impacts_of_option-complete,KSComplement,0.717114


In [119]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.933983,0.622398,0.490363
1,installation_date,number_of_legs,CorrelationSimilarity,0.922381,0.614305,0.459066
2,installation_date,number_of_piles,CorrelationSimilarity,0.940764,0.681692,0.563221
3,installation_date,height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.969232,0.599779,0.538242
4,installation_date,risk_to_other_users-complete,CorrelationSimilarity,0.933319,-0.473868,-0.340507
5,installation_date,risk_to_other_users-partial,CorrelationSimilarity,0.956859,0.828725,0.742443
6,installation_date,risk_to_personnel-complete,CorrelationSimilarity,0.846241,-0.277817,0.029701
7,installation_date,risk_to_personnel-partial,CorrelationSimilarity,0.895015,-0.274356,-0.064386
8,installation_date,impacts_of_option-complete,CorrelationSimilarity,0.938408,0.284834,0.408019
9,installation_date,technical_feasibility_or_challenge-complete,CorrelationSimilarity,0.989502,0.110643,0.131638
